# Dealing with several robots

Open the `epuck.ttt` scene as usual and copy-paste the ePuck robot several times (let's have 5 ePucks in total). When you copy-paste an object in V-REP, the copy appears at the exact same place as the original, so you have to move it a bit. If you look at the scene hierarchy, you will see that the name of the original robot is `ePuck` and the names of the copies are `ePuck#0` ... `ePuck#3`.

Connect to the simulator as usual:

In [ ]:
# necessary stuff to set the paths:
import sys
sys.path.append("..")

# Import and create the connection to the simulator:
from vrep.simulator import Simulator
simulator = Simulator()

Now let's get each of the ePucks as a Python object. To do so, we use `simulator.get_epuck()`. Without any argument (empty parentheses), it will return the first robot (the one named `ePuck`). Let's try it:

In [ ]:
epuck = simulator.get_epuck()

Let's check it is working correctly. First start the simulator:

In [ ]:
simulator.start()

Then send a command to the robot to see if it moves:

In [ ]:
epuck.rot_spd = 0.2

To get another robot, we have to tell `simulator.get_epuck()` the suffix of its name. For example, the suffix of the robot named `ePuck#0` is `"#0"`, the suffix of `ePuck#1` is `"#1"`, etc...

So, to get `ePuck#0` we write:

In [ ]:
another_epuck = simulator.get_epuck("#0")

Let's check if we can make it move as well:

In [ ]:
another_epuck.rot_spd = -0.2

Note that `simulator` maintains an list of all the requested robots (not all the robots in the scene, only those who have been requested programatically):

In [ ]:
simulator.robots

Above we see that `simulator.robots` contains two ePucks, in the order that we have requested them. Therefore, writing

In [ ]:
simulator.robots[0].rot_spd = 0.5

is equivalent to write:

In [ ]:
epuck.rot_spd = 0.5

The first notation can be useful if you want to perform the same operation on all the robots. For example, let's make them all rotate the same way:

In [ ]:
for robot in simulator.robots:
    robot.rot_spd = 2.

Try to make the other ePucks move:

In [ ]:
# necessary stuff to set the paths:
import sys
sys.path.append("..")

# Import and create the connection to the simulator:
from vrep.simulator import Simulator
simulator = Simulator()

r = simulator.get_epuck("#0")
x = simulator.get_epuck("#1")

from numpy.random import rand

r.fwd_spd = 0.05
x.fwd_spd = 0.05
 
        
# Function for moving forward:
def move_forward(epuck):
    epuck.rot_spd = 0.
    
def obstacle_avoidance(epuck):
    if epuck.min_distance_to_object("20cmHigh",group="front-left")<epuck.min_distance_to_object("20cmHigh",group="front-right"):
        epuck.rot_spd = -2
    else:
        epuck.rot_spd = 2
        
while True:    
    if r.min_distance_to_object("20cmHighPillar") < 300 :
        r.fwd_spd -= 0.001
        if r.fwd_spd < 0.03:
            r.fwd_spd = 0.03
    if x.min_distance_to_object("20cmHighPillar") < 300 :
        x.fwd_spd += 0.001
        if x.fwd_spd > 0.07:
            x.fwd_spd = 0.07
        
    if r.min_distance_to_object("ePuck")<300:
        r.fwd_spd += 0.001
        if r.fwd_spd > 0.07:
            r.fwd_spd = 0.07
    if x.min_distance_to_object("ePuck")<300:
        r.fwd_spd -= 0.001
        if r.fwd_spd < 0.03:
            r.fwd_spd = 0.03
    
    #r(hunter) detects and chases the x(prey)
    if r.min_distance_to_object("ePuck",group="front-left")<1000:
        if r.min_distance_to_object("20cmHighWall") < 200 or r.min_distance_to_object("20cmHighPillar") < 100 : 
            obstacle_avoidance(r)
        else:
            r.rot_spd = 1.
    elif r.min_distance_to_object("ePuck",group="front-right")<1000:
        if r.min_distance_to_object("20cmHighWall") < 200 or r.min_distance_to_object("20cmHighPillar") < 100 : 
            obstacle_avoidance(r)
        else:
            r.rot_spd = -1.
    else:
        if r.min_distance_to_object("20cmHighWall") < 200 or r.min_distance_to_object("20cmHighPillar") < 100 : 
            obstacle_avoidance(r)
        else:
            r.rot_spd = 0.
   
    #x(prey) detects hunter and runs away
    if x.min_distance_to_object("ePuck",group="front-left")<400:
        if x.min_distance_to_object("20cmHighWall") < 300 or x.min_distance_to_object("20cmHighPillar") < 100: 
            obstacle_avoidance(x)
        else:
            x.rot_spd = -1
    elif x.min_distance_to_object("ePuck",group="front-right")<400:
        if x.min_distance_to_object("20cmHighWall") < 300 or x.min_distance_to_object("20cmHighPillar") < 100: 
            obstacle_avoidance(x)
        else:
            x.rot_spd = 1
    elif x.min_distance_to_object("ePuck",group="rear")<400:
        if x.min_distance_to_object("20cmHighWall") < 300 or x.min_distance_to_object("20cmHighPillar") < 100: 
            obstacle_avoidance(x)
        else:
            x.rot_spd = 0.
        
        
    if r.min_distance_to_object("20cmHighWall") < 300 or r.min_distance_to_object("20cmHighPillar") < 100 : 
        obstacle_avoidance(r)
    else:
        r.rot_spd = 0.
    if x.min_distance_to_object("20cmHighWall") < 300 or x.min_distance_to_object("20cmHighPillar") < 100: 
        obstacle_avoidance(x)
    else:
        x.rot_spd = 0.
    
    
    